In [ ]:
!pip install -q transformers datasets evaluate contractions

import pandas as pd                
import numpy as np                 
import torch                       
import torch.nn as nn              
from torch.utils.data import Dataset, DataLoader  


from transformers import (
    DistilBertTokenizer,                   
    DistilBertForSequenceClassification,   
    get_scheduler,                        
    DataCollatorWithPadding,               
    DistilBertConfig                       
)

from torch.optim import AdamW              
from sklearn.metrics import accuracy_score, classification_report  
from tqdm import tqdm                      
import random                              
import re                                  
import emoji                               
import html                                
from nltk.tokenize import TweetTokenizer   
from contractions import fix as fix_contractions  
import torch.nn.functional as F            


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 8.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==1

2025-06-07 19:14:16.397203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749323656.589231      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749323656.644428      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# # 🧪 Setup & Dependencies
# Install necessary packages and import required libraries.


In [ ]:
#Set a random seed for reproducibility
SEED = 42

#Ensure reproducibility across runs for Python's built-in random module
random.seed(SEED)

#Set the seed for NumPy random number generator
np.random.seed(SEED)

#Set the seed for PyTorch CPU and GPU (if available)
torch.manual_seed(SEED)

#Ensure deterministic behavior in CUDA operations (if using GPU)
#This may slightly reduce performance but guarantees reproducibility
torch.backends.cudnn.deterministic = True



# # 🔁 Reproducibility
# Set random seeds for reproducibility.

In [ ]:
train_df = pd.read_csv("/kaggle/input/ai-2-dl-for-nlp-2025-homework-3-distil-bert/train_dataset.csv")
val_df = pd.read_csv("/kaggle/input/ai-2-dl-for-nlp-2025-homework-3-distil-bert/val_dataset.csv")
test_df = pd.read_csv("/kaggle/input/ai-2-dl-for-nlp-2025-homework-3-distil-bert/test_dataset.csv")

# # 📥 Load Data
# Read train, validation, and test datasets.

In [ ]:
# Initialize a tweet-specific tokenizer from NLTK:
# - preserve_case=False: lowercase all text
# - reduce_len=True: shorten repeated characters (e.g., "soooo" → "soo")
# - strip_handles=False: keep Twitter handles (they'll be replaced manually later)
tweet_tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=False)

#Define a tweet preprocessing function
def preprocess_tweet(text):
    #Convert emojis into textual descriptions (e.g., "😊" → " smile ")
    text = emoji.demojize(text, delimiters=(" ", " "))
    
    #Unescape any HTML entities (e.g., "&amp;" → "&")
    text = html.unescape(text)
    
    #Replace URLs with a placeholder
    text = re.sub(r"http\S+|www\.\S+", "URL", text)
    
    #Replace user mentions (e.g., @user123) with a placeholder
    text = re.sub(r"@\w+", "USER", text)
    
    #Separate hashtags from the words (e.g., "#happy" → " happy")
    text = re.sub(r"#(\w+)", r" \1", text)
    
    #Expand contractions (e.g., "don't" → "do not")
    text = fix_contractions(text)
    
    #Remove space before punctuation marks
    text = re.sub(r"\s+([?.!,])", r"\1", text)
    
    #Replace repeated punctuation (e.g., "!!!" → "!")
    text = re.sub(r"([?.!,])\1+", r"\1", text)
    
    #Reduce character repetitions (e.g., "soooo" → "soo")
    text = re.sub(r"(.)\1{2,}", r"\1\1", text)
    
    #Replace common emoticons with text labels
    emoticons = {r":\)": "smile", r":D": "laugh", r":\(": "sad", r":P": "playful", r";\)": "wink"}
    for emot, token in emoticons.items():
        text = re.sub(re.escape(emot), f" {token} ", text)
    
    #Remove non-ASCII characters
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    
    #Normalize whitespace and strip leading/trailing spaces
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

#Apply preprocessing to the training and validation datasets
train_df["CleanText"] = train_df["Text"].apply(preprocess_tweet)
val_df["CleanText"] = val_df["Text"].apply(preprocess_tweet)


# # 🧹 Preprocessing
# Define a tweet-specific text cleaning function.


In [ ]:
#Custom PyTorch Dataset class for handling tweet data
class TweetDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.tokenizer = tokenizer
        self.labels = labels
        
        #Preprocess all input texts (tweets)
        self.texts = [preprocess_tweet(t) for t in texts]
        
        #Tokenize the cleaned texts using the provided tokenizer (e.g.,DistilBERT tokenizer)
        self.encodings = self.tokenizer(
            self.texts,
            truncation=True,           #Truncate sequences longer than max_len
            padding='max_length',      #Pad sequences to the same max_len
            max_length=max_len,        #Maximum sequence length
            return_tensors='pt'        #Return PyTorch tensors
        )

    def __getitem__(self, idx):
        #Retrieve the tokenized input at index `idx`
        item = {key: val[idx] for key, val in self.encodings.items()}
        
        #Add the corresponding label (as a tensor) if labels are available
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return item

    def __len__(self):
        #Return the number of samples in the dataset
        return len(self.labels)


# # 📦 Dataset Class
# Custom dataset that tokenizes the text using DistilBERT tokenizer.

In [ ]:
#Load the pretrained DistilBERT tokenizer (lowercased version)
#This tokenizer will convert raw text into input IDs and attention masks
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

#Load the pretrained DistilBERT model for sequence classification
#num_labels=2 indicates binary classification (e.g.,positive vs.negative sentiment)
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

#Create the training dataset using the custom TweetDataset class
#Converts raw texts and labels into tokenized tensors
train_dataset = TweetDataset(
    train_df["Text"].tolist(),       #Raw tweets
    train_df["Label"].tolist(),      #Corresponding labels
    tokenizer                        #Tokenizer instance
)

#create the validation dataset similarly
val_dataset = TweetDataset(
    val_df["Text"].tolist(),
    val_df["Label"].tolist(),
    tokenizer
)

#Create the test dataset
#Labels are dummy (e.g.,all zeros) since test labels may be unknown or not used during inference
test_dataset = TweetDataset(
    test_df["Text"].tolist(),
    [0]*len(test_df),                #Placeholder labels
    tokenizer
)

#Create a data collator to dynamically pad inputs in a batch
#Ensures that all sequences in a batch are padded to the length of the longest one
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# # 🤖 Model and Tokenizer
# Load the pre-trained DistilBERT model and tokenizer.

In [ ]:
batch_size = 8 #Best value after optuna
learning_rate = 1.4647207744407194e-05#Best value after optuna
num_epochs = 2#Best value after optuna

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=data_collator)
val_loader = DataLoader(val_dataset, batch_size=batch_size, collate_fn=data_collator)
test_loader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=data_collator)


# # 🛠️ Hyperparameters & Dataloaders
# Define training parameters and construct dataloaders.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)
num_training_steps = num_epochs * len(train_loader)
lr_scheduler = get_scheduler("cosine_with_restarts", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# # ⚙️ Training Setup
# Prepare optimizer, learning rate scheduler, and move model to GPU.

In [ ]:
#Initialize tracking variables
best_val_acc = 0
best_model_state = None
train_losses, val_losses, val_accuracies = [], [], []
val_probs, val_targets = [], []

#Training loop
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    #Validation loop
    model.eval()
    all_labels, all_preds, all_probs = [], [], []
    total_val_loss = 0

    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            logits = outputs.logits

            total_val_loss += loss.item()

            #Get prediction probabilities and labels
            probs = F.softmax(logits, dim=1)[:, 1]
            preds = torch.argmax(logits, dim=1)

            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch["labels"].cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    acc = accuracy_score(all_labels, all_preds)
    val_losses.append(avg_val_loss)
    val_accuracies.append(acc)
    val_probs = all_probs
    val_targets = all_labels

    #Print results for this epoch
    print(f"[DistilBERT] Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}, Accuracy = {acc:.4f}")
    print(classification_report(all_labels, all_preds))

    #Save the best model state
    if acc > best_val_acc:
        best_val_acc = acc
        best_model_state = model.state_dict()


# # 🎯 Training Loop
# Perform training with validation and checkpointing.

In [ ]:
def extract_cls_embeddings_distilbert(model, dataloader):
    model.eval()
    embeddings = []
    labels = []
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model.distilbert(input_ids=input_ids, attention_mask=attention_mask)
            cls_embeds = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(cls_embeds)
            if 'labels' in batch:
                labels.extend(batch['labels'].cpu().numpy())
    embeddings = np.vstack(embeddings)
    labels = np.array(labels)
    return embeddings, labels

distilbert_embeddings, distilbert_labels = extract_cls_embeddings_distilbert(model, val_loader)


# # 🔍 Extract CLS Embeddings
# For visualization tasks like t-SNE.

In [ ]:
model.load_state_dict(best_model_state)
model.eval()

all_preds = []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        preds = torch.argmax(outputs.logits, dim=1)
        all_preds.extend(preds.cpu().numpy())

test_df["Label"] = all_preds
test_df[["ID", "Label"]].to_csv("submission.csv", index=False)
print("✅ DistilBERT submission saved.")


# # 📝 Save Submission
# Generate predictions for the test set and save them.

----------------------------------------------------------

# CODE FOR ALL THE GRAPHS 

In [ ]:
# ==================== Imports ====================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, classification_report, roc_curve, auc, confusion_matrix
from wordcloud import WordCloud
from textblob import TextBlob
import emoji
from collections import Counter


## 📦 Imports

Load essential Python libraries for data handling, visualization, evaluation, and sentiment analysis.


In [ ]:
# ==================== Data Variables ====================
y_train = train_df["Label"].values
y_val = val_df["Label"].values


## 🧾 Prepare Label Variables

Extract target labels for training and validation sets.


In [ ]:
# ==================== t-SNE on DistilBERT ====================
tsne = TSNE(n_components=2, perplexity=30, learning_rate=200, n_iter=1000, random_state=42)
distilbert_tsne_embeds = tsne.fit_transform(distilbert_embeddings)

plt.figure(figsize=(10, 6))
colors = ['red' if l == 0 else 'green' for l in distilbert_labels]
plt.scatter(distilbert_tsne_embeds[:, 0], distilbert_tsne_embeds[:, 1], c=colors, alpha=0.6)
plt.title("t-SNE Visualization of DistilBERT [CLS] Embeddings")
plt.xlabel("t-SNE Dim 1")
plt.ylabel("t-SNE Dim 2")
plt.legend(handles=[
    plt.Line2D([0], [0], marker='o', color='w', label='Negative', markerfacecolor='red', markersize=8),
    plt.Line2D([0], [0], marker='o', color='w', label='Positive', markerfacecolor='green', markersize=8)
])
plt.grid(True)
plt.show()


## 🔍 t-SNE Visualization of DistilBERT Embeddings

Visualize high-dimensional DistilBERT `[CLS]` embeddings using t-SNE to explore separability between positive and negative classes.


In [ ]:
# ==================== Word Cloud ====================
def plot_wordcloud(text_series, title, filename=None):
    text = ' '.join(text_series.dropna().astype(str))
    wc = WordCloud(width=800, height=400, background_color='white').generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(title)
    if filename:
        plt.savefig(filename, bbox_inches='tight')
    plt.tight_layout()
    plt.show()

plot_wordcloud(train_df['Text'], 'Word Cloud (Raw Tweets)', 'raw_wc.png')
plot_wordcloud(train_df['CleanText'], 'Word Cloud (Cleaned Tweets)', 'clean_wc.png')


## ☁️ Word Clouds: Raw vs Cleaned Tweets

Visualize the most frequent words in raw and preprocessed tweets using word clouds.


In [ ]:
# ==================== Length Distribution ====================
def plot_length_distribution(raw_texts, clean_texts, filename=None):
    raw_lengths = raw_texts.dropna().astype(str).str.split().apply(len)
    clean_lengths = clean_texts.dropna().astype(str).str.split().apply(len)

    plt.figure(figsize=(10, 5))
    plt.hist(raw_lengths, bins=50, alpha=0.6, label='Raw', color='red')
    plt.hist(clean_lengths, bins=50, alpha=0.6, label='Clean', color='green')
    plt.xlabel("Text Length (Tokens)")
    plt.ylabel("Frequency")
    plt.title("Text Length Distribution: Raw vs Clean")
    plt.legend()
    if filename:
        plt.savefig(filename)
    plt.tight_layout()
    plt.show()

plot_length_distribution(train_df['Text'], train_df['CleanText'], 'text_len_dist.png')


## 📏 Text Length Distribution

Compare token length distributions of raw and cleaned tweets.


In [ ]:
# ==================== Emoji Frequency ====================
def extract_emojis(text):
    return [char for char in text if char in emoji.EMOJI_DATA]

def plot_emoji_freq(texts, top_n=10, filename=None):
    all_emojis = []
    for text in texts.dropna().astype(str):
        all_emojis.extend(extract_emojis(text))

    top_emojis = Counter(all_emojis).most_common(top_n)
    if not top_emojis:
        print("No emojis found.")
        return

    labels, values = zip(*top_emojis)

    plt.figure(figsize=(8, 4))
    plt.bar(labels, values, color='orange')
    plt.title("Top Emojis in Raw Tweets")
    plt.ylabel("Frequency")
    if filename:
        plt.savefig(filename)
    plt.tight_layout()
    plt.show()

plot_emoji_freq(train_df['Text'], top_n=10, filename='emoji_freq.png')


## 😊 Top Emoji Frequency

Display the top 10 most used emojis in raw tweets.


In [ ]:
# ==================== Sentiment Polarity Distribution ====================
def plot_sentiment_distribution(texts, filename=None):
    texts = texts.dropna().astype(str)
    polarities = [TextBlob(text).sentiment.polarity for text in texts]

    plt.figure(figsize=(10, 5))
    plt.hist(polarities, bins=50, color='blue', alpha=0.7)
    plt.xlabel("Polarity")
    plt.ylabel("Frequency")
    plt.title("Sentiment Polarity Distribution (TextBlob)")
    if filename:
        plt.savefig(filename)
    plt.tight_layout()
    plt.show()

plot_sentiment_distribution(train_df['Text'], filename='sent_pol_raw.png')
plot_sentiment_distribution(train_df['CleanText'], filename='sent_pol_clean.png')


## 📊 Sentiment Polarity Distribution

Plot sentiment polarity distribution (range -1 to 1) using TextBlob for both raw and cleaned text.


In [ ]:
# ==================== ROC Curve ====================
fpr, tpr, _ = roc_curve(val_targets, val_probs)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {roc_auc:.4f})", color='darkorange')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Validation ROC Curve")
plt.legend()
plt.grid(True)
plt.show()


## 🧪 ROC Curve on Validation Set

Plot the ROC curve and calculate AUC for binary classification performance.


In [ ]:
# ==================== Training Curve ====================
plt.figure()
plt.plot(train_losses, label='Train Loss', color='blue')
plt.plot(val_losses, label='Validation Loss', color='green')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training & Validation Loss")
plt.legend()
plt.grid(True)
plt.show()


## 📉 Training & Validation Loss Curve

Visualize training and validation loss to detect overfitting or underfitting during training.


In [ ]:
# ==================== Confusion Matrix ====================
val_probs = np.array(val_probs)
val_preds = (val_probs >= 0.5).astype(int)
cm = confusion_matrix(val_targets, val_preds)
labels = ['Negative', 'Positive']

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix on Validation Set')
plt.tight_layout()
plt.show()


## ✅ Confusion Matrix

Visualize how well the classifier distinguishes between the classes using the confusion matrix.
